In [ ]:
# 출처 : https://wikidocs.net/233344
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [9]:
# 구조화된 출력 체인(Structured Output Chain)
# 특정 주제에 대한 5지선다형 퀴즈와 정답를 생성하는 과정을 구현합니다.
    
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# Quiz 클래스는 퀴즈의 질문, 난이도, 그리고 5개의 선택지와 정답를 정의합니다.
class Quiz(BaseModel):
    question:str = Field(..., description="퀴즈의 질문")
    level:str = Field(..., description="퀴즈의 난의도를 나타냅니다.(쉬움, 보통, 어려움)")
    select:List[str] = Field(..., description="퀴즈의 5개 선택지 입니다.")
    answer:str = Field(..., description="5개 선택지 중 정답을 나타냅니다.")

# 모델 생성
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

# 프롬프트 설정
# ChatPromptTemplate는 퀴즈 생성을 위한 대화형 프롬프트를 정의합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a world-famous quizzer and generates quizzes in structured formats."
        ),
        (
            "human",
            "TOPIC 에 제시된 내용과 관련한 5지선다형 퀴즈를 출제해 주세요."
            "한국어로 퀴즈를 만들어주세요."
            "만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요."
            "정답도 알려주세요."
            "단, 문제에 TOPIC 에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}"
        ),
    ]
)

# 체인 생성
# create_structured_output_runnable 함수는 이러한 구성 요소를 결합하여 
# 주어진 주제에 대한 퀴즈를 생성하는 실행 가능한 체인을 만듭니다.
chain = create_structured_output_runnable(Quiz, llm, prompt)

# 예측
response = chain.invoke({"topic":"엑셀(Excel) 자격증 시험"})
print(response)


question='엑셀 자격증 시험에서 사용되는 함수 중 데이터를 찾아내는 함수는 무엇인가요?' level='보통' select=['VLOOKUP', 'SUM', 'IF', 'AVERAGE', 'COUNTIF'] answer='VLOOKUP'
